# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import panel as pn  # 
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 
code=r""" def calculate_annual_salary(monthly_salary):
    # Assuming a standard 12-month work year
    months_in_year = 12

    # Calculate annual salary
    annual_salary = monthly_salary * months_in_year

    return annual_salary

# Example usage
try:
    # Get monthly salary input from the user
    monthly_salary = float(input("Enter your monthly salary: $"))

    # Ensure the input is a positive number
    if monthly_salary < 0:
        raise ValueError("Monthly salary should be a positive number.")

    # Calculate and print the annual salary
    annual_salary = calculate_annual_salary(monthly_salary)
    print(f"Your annual salary is: ${annual_salary:.2f}")

except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
"""
context = [ {'role':'system', 'content':"""

Hi you re a senior developer and there is a junior developer who has to write fucntions 
in the already existing code, 
the may have questions about the structure of teh code and the template of it 
understanding logic or making their own for the function 
Some issues regarding company code conventions 
This is the code input from company code base {code}
Help the junior developer!!
Let them ask questions to you in context of the code 

"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard